## **Tagging & Extracting using LLM's, LangChain & Pydantic**

We can use **LangChain & Pydantic** in order to get stable outputs from LLMs.

In this hypothetical scenario, let's assume we need to perform a number of tasks on a text extract, for example a complaint.

Imagine that we want to know:
- A series of customer details
- Any particular entities mentioned
- A summary of the complaint
- A sentiment

We can define the expected outputs as Pydantic classes.

In [1]:
import os
import openai

from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain.chat_models import ChatOpenAI

from template import TEMPLATE
from models import Overview
from utils import build_prompt, Chain

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file
openai.api_key = os.environ["OPENAI_API_KEY"]

### **Step-By-Step**


I have defined structured Pydantic models (models.py). These will form the basis of our structured outputs.

The next steps I go through are:

1. Building the prompt (using template.py)
2. Initiating the OpenAI model
3. Convering my Pydantic class (which itself contains several other classes) in to an OpenAI function
4. Bind that OpenAI function to the model
5. Create a chain of events (the prompt, the binded function & model, and a helper parser)

In [2]:
prompt = build_prompt(TEMPLATE)

model = ChatOpenAI(temperature=0)

overview_tagging_function = [convert_pydantic_to_openai_function(Overview)]
tagging_model = model.bind(
    functions=overview_tagging_function, function_call={"name": "Overview"}
)
chain = prompt | tagging_model | JsonOutputFunctionsParser(key_name="Overview")

### **Hypothetical Complaint**

Here we have a sample complaint which contains numerous pieces of information.

In [3]:
complaint = """
My name is Joe Blogs. I'm 21 yrs old, and as I am a civil engineer by trade, I can see some issues that I am not happy about.
I'm not happy that the engineer turned up late.
The pipe outside my house burst and it made a right mess. My friend Santos was furious, and he is 80.
"""

In [4]:
chainObject = Chain(chain)
chainObject.run(complaint)

{'summary': 'Joe Blogs, 21 yrs old civil engineer, unhappy about engineer being late and pipe burst outside his house',
 'sentiment': {'sentiment': 'Negative'},
 'keywords': [{'word': 'Joe Blogs'},
  {'word': '21 yrs old'},
  {'word': 'civil engineer'},
  {'word': 'engineer'},
  {'word': 'late'},
  {'word': 'pipe burst'},
  {'word': 'house'},
  {'word': 'mess'},
  {'word': 'friend'},
  {'word': 'Santos'},
  {'word': 'furious'},
  {'word': '80'}],
 'people': {'people': [{'name': 'Joe Blogs',
    'age': 21,
    'profession': 'civil engineer'},
   {'name': 'Santos', 'age': 80, 'profession': None}]},
 'intensity': '3'}

As the output is a dictionary, we can access each element easily.